# A basic Recommender System for Different Genre of Movies Based on User Ratings:

We are going to use the "MoviesLes Dataset" to build a recommender system. The data set contains over 20 milion of ratings for different movies. 

## Data Analysis:

We are going to use the movielens to build a really simple "item similarity based" recommender system. 

In [5]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

Next we load the dataset using pandas funtion called read_csv(). The dataset is "tab" seperated so we need to pass in "\t" to the sep parameter of the "read_csv()" function. We then pass in the column names using the "names" parameter.

In [6]:
df = pd.read_csv('film_ratings.csv', sep = '\t', names = ['user_id', 'item_id', 'rating', 'timestamps'])
df.head() # checking the head of the datatosee the data we are dealing with

At this point in our dataset we have the movie ids instead of their names. It would be awesome if we could map the movie ids with their names. Fortunately we have the data for that and we can merge the two datasets (The above dataset which would include the movie-ids) with the movie names using a mutual key (column) called item_id. Let's do it:

In [ ]:
movie_titles = pd.read_csv("Movie_titles")
movie_titles.head()

# Merging the two datasets using a mutual key (item-id)
df = pd.merge(df, movie_titles, on = "item_id")
df.head()

Now in the dataset we would have the current features (attributes):
- user_id: The Id of the user who rated the movie
- item_id: The Id of the movie
- rating: The rating the user gave the movie, between 1 and 5
- timestamp: The time the movie was rated
- title: The title of the movie

Using "describe" or "info" methods we can get a brief description of our dataset. This is important in order to enable us understand the dataset we are working with.

In [ ]:
df.describe()

From the above description we can tell that:
- The average rating is 3.52
- The max rating is 5
- The dataset has close to 100003 records

We now create a dataframe with the average rating for each movie and the number of ratings. We are going to use these ratings to calculate the correlation between the movies later. 

- Correlation: Statistical measure which indicates the extent to which two or more variables fluctuate together.
- Movies that have high correlation coefficient, are the most similar to each other.

In our case we use "Pearson Correlation". This number will lie between -1 and 1. 1 indicates a positve linear correlation while -1 indicates a negative correlation. 0 indicates no linear correlation. therefor movies with a 0 correlation are not similar at all. 

We group the dataset by the "title" column and compute its mean to obtain the average rating for each movie. 

In [ ]:
# Rating for each movie based on their title 
rating = pd.DataFrame(df.groupby("title")['rating'].mean())
rating.head()

As the next step we would like to see the number of ratings for each movie. We do this by creating a "number_of_ratings"column. This is important so that we can see the relationship between the average rating of a movie and the number of ratings the movie got. It is very possible that a 5 star movie was rated by just one person. It is therefor statistically incorrect to classify that movie has a 5 star movie. We would therefor need to set a threshold for the minimum number of ratings as we build the recommender system. In order to create this new column we use "pandas" group by utility. We group by the title column and then "count" the number of ratings for each of those titles. Afterwards we view the new dataframe by using the "head()" function.  

In [ ]:
ratings['number_of_ratings'] = df.groupby('title')['rating'].count()
ratings.head()

We now plot a histogram using pandas plotting functionality to visualize the distribution of the ratings.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ratings['tating'].hist(bins = 50)

We can see that most of the movies are rated between 2.5 and 4. Next let's visualize the number_of_ratings column in a similar manner.

In [ ]:
ratings['number_of_ratings'].hist(bins = 50)

From the above histogram it is clear that most movies have few ratings. Movies with most ratings are those that are most famous.

Let's now check the relationship between the rating of a movie and the number of ratings. We do this by plotting a scatter plot using "seaborn". Seaborn enables us to do this using "jointplot() function.

In [ ]:
import seaborn as sns
sns.jointplot(x = 'rating', y = 'number_of_ratings', data=ratings)

From the scatter plot one can see that there is a positve relationship between the average rating of a movie and the number of ratings. The graph indicates that the more ratings a movie gets, the higher the average rating will be. This is important to note when we want to set a threshold for the number of ratings a movie has to do more analysis. 

## Item Based Recommender System

Now we move to the next step and we create a simple item-based recommender system. In order to do that we need to convert our dataset into a "matrix" with the movie  titles as columns, the user_id as index, and rhe ratings as the values. By doing this we shall get a dataframe with the columns as the movie titles and the rows as the user-ids. Each column represents all the ratings of a movie by all users. The rating appear as NAN where a user didn't rate a certain movie. We shall use this matrix to compute the correaltion between the ratings of a single movie and the rest of the movies in the matrix. We use pandas 'pivot_table' utility to create the movie matrix. 

In [ ]:
movie_matrix = df.pivot_table(index='user_id', columns='title', values='rating')
movie_matrix.head()

Next we look at the most rated movies and choose two of them to work with in this simple recommender system. We use pandas 'sort_values' function and set 'ascending' to false in order to arrange the movies from the most rated. We then use the head() function to view the top 10.

In [ ]:
ratings.sort_values('number_of_ratings', ascending=Fale).head(10)

Let's Assume that a user has watched Air Force One (1997) and Contact (1997). We would like to recommend movies to this user based on this watching history. The goal is to look for movies that are similar to Contact and AirForce One. We can achieve this goal by computing the correaltion between these two movie ratings and the ratings of the rest of the movies in the dataset. The first step is to create a dataframe with the ratings of these movies from our "movie_matrix"

In [ ]:
Airforce_user_rating = movie_matrix['Air Force One (1997)']
Contact_user_rating = movie_matrix['Contact (1997)']

We now have the dataframe showing the user_id and the rating they gave the two movies. Let's take a look at them below. Let's take a look at them below:

In [ ]:
Airforce_user_rating.head()
Contact_user_rating.head()

In order to compute the correlation between two dataframes we can use pandas "corrwith" function. "corrwith" computes the pairwise correlation of rows or columns of two dataframe objects. Let's use this funciton to get the correlation between each movie's rating and the ratings of the Air Force One movie. 

In [ ]:
similar_to_air_force_once = movie_matrix.corrwith(Airforce_user_rating)

We can see that the correaltion between Air Force One movie and Til There Was You (1997) is 0.87. This indicates a very strong similarities (in terms of ratings) between these two movies. 

In [ ]:
similar_to_air_force_one.head()

Let's move on and compute the correlation between Contact (1997) ratings and the rest of the movie ratings. The procedure is exactly the same as what we did above. 

In [ ]:
similar_to_contact = movie_matrix.corrwith(contact_user_rating)

We realize that there is a strong correlation, 0.904, between Contact (1997) and Til There Was You (1997).

In [ ]:
similar_to_contact.head()

Since our initial movie_matrix has lots of NAN or missing values, we now drop those missing values and transform correlation results into dataframes to make the results look more appealing.

In [ ]:
corr_contact = pd.DataFrame(similar_to_contact, columns=['Correlation'])
corr_contact.dropna(inplace=True)
corr_contact.head()

corr_air_force = pd.DataFrame(similar_to_air_force, columns=['Correlation'])
corr_air_force.dropna(inplace=True)
corr_air_force.head()

These two dataframes above show us the movies that are most similar to Contact (1997) and Air Force One (1997) respectively. One challenge here is the fact that some of the movies have a very few ratings and may end up being recommended simply because one or two people gave them a 5 star rating. We can fix this by setting a threshold for the number of ratings. From the histogram earlier we saw a sharp decline in the number of ratings from 100. We shall therefor set this as the threshold, however this is a number you can play around until you get a suitable option. In order to do this we need to join the two dataframes with the 'number_of_ratings' column in the rating data frame.

In [ ]:
corr_air_force = corr_air_force.join(ratings['number of ratings'])
corr_contact = corr_contact.join(ratings['number of ratings'])
corr_air_force.head()
corr_contact.head()

We now obtain movies that are most similar to Air Force One (1997) by limiting them to movies that have at least 100 reviews. We then sort them by the correlation column and view the top 10.

In [ ]:
corr_air_force[corr_airforce['number_of_ratings'] > 100].sort_values(by='correlation', ascending = False).head(10)
corr_contact[corr_contact['number_of_ratings'] > 100].sort_values(by='correlation', ascending  =False).head(10)

## How to improve the recommender system?

This system can be improved by building a Memory-Based Collaborative Filtering based system. In this case, we'd divide the data into a training set and a test set. We'd then use techniques such as "Cosine similarity" to compute the similarity between the movies. An alternative is to build a Model-Based Collaborative Filtering System. This is based on "Matrix Factorization". Matrix Factorization is good at dealing with scalability and sparsity than the former. You can then evaluate your model using techniques such as Root Mean Squared Error (RMSE)

Some of the algorithms that are being used to build advanced recommener systems include:
- "Auto-Encoders" 
- "Restricted Boltzmann Machines"